In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

In [ ]:
#chack for updated files\n,
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

#import packages
from __future__ import division

import numpy as np
import glob, json
import pickle
import os as os_pack
import matplotlib.pyplot as plt
import corner
#%matplotlib inline\n",
%config InlineBackend.figure_format = 'retina'
plt.rc('text', usetex=True)
plt.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]
import healpy as hp
import os, glob, json, pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as sl
import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const
from enterprise_extensions import blocks
from enterprise_extensions import models as ee_models
from enterprise_extensions import model_utils as ee_model_utils
from enterprise_extensions import model_orfs
from enterprise_extensions.frequentist import optimal_statistic as opt_stat
from enterprise_extensions import sampler as ee_sampler
from enterprise.signals.signal_base import LogLikelihood
import enterprise_wavelets as models
from enterprise.signals.deterministic_signals import Deterministic
from enterprise.signals.parameter import function
from la_forge.core import Core
from la_forge.diagnostics import plot_chains
from la_forge import rednoise
import la_forge
import corner
import h5py
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
import re
#style
import cProfile
    
#import Fast_Burst_likelihood as FB

# Testing likelihood in notebook with bad samples

## Copying PTA generation from QuickBurst_MCMC

# 7/7 - Incomplete PTA generation/likelihood testing for bad samples from previous chain. 
A bit more complicated, since we need to make sure the generated PTA matches the one we are using in our likelihood calls, as well as ensuring that the jumps match (will need possible if statements for each kind of jump (and copy likelihood calls out of QuickBurst_MCMC for each jump).
# I may be overcomplicating this. 

Template: 

- create pta (complicated part -> needs to match chain setup)
- create likelihood objects for each chain
- calculate likelihood from each object using bad samples
- check which steps are being taken while going through samples


# 7/17/23 Edit: I was indeed OVERCOMPLICATING things...

In [ ]:
########## 2 wavelet dataset
# parDir =  '/home/reyna/BayesHopperBurst/BenceData/2_Wavelet_Sim/par/'
# timDir =  '/home/reyna/BayesHopperBurst/BenceData/2_Wavelet_Sim/2_wavelet_tims/'
# psrlist = sorted(glob.glob(parDir+'*.par'))
# for _ in range(len(psrlist)):
#     psrlist[_] = re.sub(parDir, '', psrlist[_])
#     psrlist[_] = re.sub('_NANOGrav_12yv2.gls.par', '', psrlist[_])
# parfiles = sorted(glob.glob(parDir+'*.par'))
# timfiles = sorted(glob.glob(timDir+'*.tim'))
# psrs_sim = []
# for p, t in zip(parfiles, timfiles):
#     psr = Pulsar(p, t, ephem=None, clk=None)
#     psrs_sim.append(psr)

########## 12.5 year Astro4Cast dataset
with open("/home/reyna/BayesHopperBurst/QuickBurst/12p5_year_A4Cast/quickburst_test_12p5yrAstro4Cast_parabolic_3Mpc_60M_psrs.pkl", 'rb') as f:
    psrs_sim = pickle.load(f)

In [ ]:
#Loading in pickle and noise files
################# 2 wavelet dataset
#noise_file_sim = "/home/reyna/BayesHopperBurst/BenceData/with_burst/params_simulated.json"
################# 12.5 year Astro4cast
noise_file_sim = "/home/reyna/BayesHopperBurst/QuickBurst/12p5_year_A4Cast/quickburst_test_12p5yrAstro4Cast_parabolic_3Mpc_60M/channelized_12p5yr_v3_full_noisedict.json"

#psrlist = np.loadtxt('/home/reyna/15yr_v1p0/15yr_v1-20211001T235643Z-001/15yr_v1/psrlist_15yr_pint.txt', dtype = str)
with open(noise_file_sim, 'r') as h:
    noise_params_sim = json.load(h)
#Temporary to get code to not crash
psrs_sim = psrs_sim[0:]
psrlist = [psr.name for psr in psrs_sim]
print(psrlist)
for i in range(len(psrs_sim)):
    print((max(psrs_sim[i].toas) - min(psrs_sim[i].toas))/(3.17*10**(7)))

In [ ]:
noise_dict_sim = {} #new noise dict to conver the simulated noise file into a readable formate
for k,v in noise_params_sim.items():
    if '_equad' in k:
        noise_dict_sim.update({k.split('_equad')[0] + '_t2equad': v})
    else:
        noise_dict_sim.update({k : v})

### Set number of chains , tau scan and glitch tau scan file paths

In [ ]:
############# 12.5 year Astr4cast
ts_file = "/home/reyna/BayesHopperBurst/QuickBurst/12p5_year_A4Cast/tauscan_12p5yr-A4Cast_like_real_Allnoise_3MPc.pkl"
glitch_ts_file = "/home/reyna/BayesHopperBurst/QuickBurst/12p5_year_A4Cast/glitch_tauscan_12p5yr-A4Cast_like_real_Allnoise_3MPc.pkl"

############## 2 wavelet dataset
# ts_file = "/home/reyna/BayesHopperBurst/BenceData/2_Wavelet_Sim/tauscan_2_wavelet_dataset.pkl"
# glitch_ts_file = "/home/reyna/BayesHopperBurst/BenceData/2_Wavelet_Sim/tauscan_2_wavelet_dataset.pkl.pkl"

### Copy run settings from run you'd like to test against enterprise

In [ ]:
#Setting dataset max time and reference time
maximum = 0
minimum = np.inf
for psr in psrs_sim:
    if psr.toas.max() > maximum:
        maximum = psr.toas.max()
    if psr.toas.min() < minimum:
        minimum = psr.toas.min()


#Sets reference time
tref = minimum

t0_max = (maximum - minimum)/365/24/3600
print(t0_max)

In [ ]:
#Various parameters used to generate PTA (copy from run you want to test with)
#(for keeping PTA the same in testing setting)
max_n_wavelet=2
min_n_wavelet=0
n_wavelet_start=1
RJ_weight=2
glitch_RJ_weight=2
regular_weight=4
noise_jump_weight=2
PT_swap_weight=2
tau_scan_proposal_weight=2
glitch_tau_scan_proposal_weight=2
tau_scan_file=ts_file
glitch_tau_scan_file=glitch_ts_file
#gwb_log_amp_range=[-18,-15]
rn_log_amp_range=[-18,-15]
wavelet_log_amp_range=[-10.0,-5]
per_psr_rn_log_amp_range=[-18,-11]
prior_recovery=False
#gwb_amp_prior='log-uniform'
rn_amp_prior='log-uniform'
wavelet_amp_prior='uniform'
per_psr_rn_amp_prior='log-uniform'
#gwb_on_prior=0.975
max_n_glitch=2
n_glitch_start=1
glitch_log_amp_range=[-10.0,-5]
glitch_amp_prior='uniform'
t0_max=t0_max
tref = tref
draw_from_prior_weight=0
de_weight=0
vary_white_noise=False
include_rn=True
vary_rn=True
include_equad=True
include_ecorr=True
wn_backend_selection=True
noisedict=noise_dict_sim
include_per_psr_rn=True
vary_per_psr_rn=True
ent_lnlike_test=True

In [ ]:
max_n_wavelet=3
min_n_wavelet=0
n_wavelet_start=1
RJ_weight=2
glitch_RJ_weight=2
regular_weight=4
noise_jump_weight=2
PT_swap_weight=2
tau_scan_proposal_weight=2
glitch_tau_scan_proposal_weight=2
tau_scan_file=ts_file
glitch_tau_scan_file=glitch_ts_file
#gwb_log_amp_range=[-18,-15]
rn_log_amp_range=[-18,-15]
wavelet_log_amp_range=[-10.0,-5]
per_psr_rn_log_amp_range=[-18,-11]
prior_recovery=False
#gwb_amp_prior='log-uniform'
rn_amp_prior='log-uniform'
wavelet_amp_prior='uniform'
per_psr_rn_amp_prior='log-uniform'
#gwb_on_prior=0.975
max_n_glitch=3
n_glitch_start=1
glitch_log_amp_range=[-10.0,-5]
glitch_amp_prior='uniform'
t0_max=t0_max
tref = tref
draw_from_prior_weight=0
de_weight=0
vary_white_noise=True
include_rn=True 
vary_rn=True
include_equad=True
include_ecorr=False
wn_backend_selection=True
noisedict=noise_dict_sim
include_per_psr_rn=True
vary_per_psr_rn=True
#resume_from=savefile
#per_psr_rn_start_file=RN_start_file
save_every_n=100
n_fast_to_slow=10
thin = 1
ent_lnlike_test = True

### Creating PTA objects for QB and enterprise 

In [ ]:
import QuickBurst_MCMC_testing as QB_MCMC
import QB_FastPrior

#PTA generation w/ 
pta, ent_ptas, QB_FP, QB_FPI, glitch_indx, wavelet_indx, per_puls_indx, rn_indx, num_per_puls_param_list = QB_MCMC.get_pta(psrs_sim, vary_white_noise=vary_white_noise, include_equad=include_equad,
                                                                                                    include_ecorr = include_ecorr, wn_backend_selection=wn_backend_selection,
                                                                                                    noisedict=noisedict, include_rn=include_rn, vary_rn=vary_rn,
                                                                                                    include_per_psr_rn=include_per_psr_rn, vary_per_psr_rn=vary_per_psr_rn,
                                                                                                    max_n_wavelet=max_n_wavelet, efac_start = None, rn_amp_prior=rn_amp_prior,
                                                                                                    rn_log_amp_range=rn_log_amp_range, rn_params=[-13.0,1.0], per_psr_rn_amp_prior=per_psr_rn_amp_prior,
                                                                                                    per_psr_rn_log_amp_range=per_psr_rn_log_amp_range, gwb_amp_prior='uniform',
                                                                                                    gwb_log_amp_range=[-18, -11], wavelet_amp_prior=wavelet_amp_prior,
                                                                                                    wavelet_log_amp_range=wavelet_log_amp_range, prior_recovery=prior_recovery, ent_lnlike_test = ent_lnlike_test,
                                                                                                    max_n_glitch=max_n_glitch, glitch_amp_prior=glitch_amp_prior, glitch_log_amp_range=glitch_log_amp_range,
                                                                                                    t0_min=0.0, t0_max=t0_max, f0_min=3.5e-9, f0_max=1e-7,
                                                                                                    TF_prior=None, tref=tref)

In [ ]:
[ent_ptas[n_wavelet_start][n_glitch].params for n_glitch in range(max_n_glitch+1)]

### Loading in chain and step_array 

In [ ]:
#filepath = "/home/reyna/BayesHopperBurst/QuickBurst/Testing/2_wavelet_sim/convergencetest/enterprise_comparison/3w3g_allnoise_alljumps_on"
#filepath = "/home/reyna/BayesHopperBurst/QuickBurst/Testing/2_wavelet_sim/bugtesting/enterprise_comparison_take2/2w2g_allnoisevaried_alljumps_newLike_oldMCMC_enterprisetest_take2"
#filepath = "/home/reyna/BayesHopperBurst/QuickBurst/Testing/2_wavelet_sim/bugtesting/enterprise_comparison_take2/2w2g_allnoisevaried_CovCalc_alljumps_oldLike_oldMCMC_enterprisetest_take5"
#filepath = "/home/reyna/BayesHopperBurst/QuickBurst/Testing/2_wavelet_sim/bugtesting/enterprise_comparison_take2/2w2g_allnoisevaried_CovCalc_SigmaNotAppend_alljumps_oldLike_oldMCMC_enterprisetest"
#filepath = "/home/reyna/BayesHopperBurst/QuickBurst/Testing/2_wavelet_sim/bugtesting/enterprise_comparison_take2/2w2g_allnoisevaried_CovCalc_rever_logdet_alljumps_oldLike_oldMCMC_enterprisetest"
#filepath = "/home/reyna/BayesHopperBurst/QuickBurst/12p5_year_A4Cast/chains/12p5yr_A4Cast_2g2w_nointWN_dictchange_oldlike_oldMCMC_enterprisetest"
#filepath = "/home/reyna/BayesHopperBurst/QuickBurst/Testing/2_wavelet_sim/bugtesting/enterprise_comparison_take2/3w3g_allnoisevaried_CovCalc_UnJit_alljumps_oldMCMC_enterprisetest"
filepath = "/home/reyna/BayesHopperBurst/QuickBurst/12p5_year_A4Cast/chains/12p5yr_A4Cast_2g2w_nointWN_CovCalc_UnJit_oldMCMC_enterprisetest"
chain_filepath = filepath + '.h5df'
with h5py.File(chain_filepath, 'r+') as f:
    n_chain = 2
    samples_array = f['samples_cold'][()]
    log_likelihood = f['log_likelihood'][()]
    ent_lnlikelihood = f['ent_lnlikelihood'][()]
    acc_frac = f['acc_fraction'][()]
    param_names = f['par_names'][()]

print(acc_frac)
#print(ent_lnlikelihood)
samples = samples_array

In [ ]:
n_wavelet = int(samples[0,1,0])
n_glitch = int(samples[0,1,1])
temp_entlike1 = ent_ptas[n_wavelet][n_glitch].get_lnlikelihood(QB_MCMC.remove_params(samples[0,1,2:], 0, 0, wavelet_indx, glitch_indx, n_glitch = n_glitch, n_wavelet = n_wavelet, max_n_glitch = max_n_glitch, max_n_wavelet = max_n_wavelet, params_slice = True))

In [ ]:
del ent_ptas[n_wavelet][n_glitch]

In [ ]:
ent_ptas[n_wavelet][n_glitch].items()[0]

In [ ]:
d0 = dict((k,v) for k,v in zip(pta.param_names, samples[0, 0, 2:]))
d0

In [ ]:
TNT = pta.get_TNT(d0)
phiinvs = pta.get_phiinv(d0, logdet = True, method = 'partition')

In [ ]:
np.shape(TNT[0])

In [ ]:
phiinv, logdet = phiinvs[1]
np.shape(phiinv)

### Getting samples that don't agree with enterprise

In [ ]:
# Getting bad samples from samples_array
bad_samples = []
idxs = []
n_chain = [n for n in range(len(acc_frac[6]))]
starting_indx = 0
ending_indx = int(len(log_likelihood[0])/1)
for n in n_chain:
    print(n)
    np.max(np.exp(log_likelihood[n][:] - ent_lnlikelihood[n][:]))
    temp_array = []
    temp_idxs = []
    temp_samp_array = []
    for i in range(starting_indx, ending_indx):
        if (np.exp(log_likelihood[n][i] - ent_lnlikelihood[n][i]) > 1+1e-6 ):#and np.exp(log_likelihood[n][i] - ent_lnlikelihood[n][i]) < 1+1e-11):
            temp_array.append((log_likelihood[n][i] - ent_lnlikelihood[n][i]))
            temp_samp_array.append(samples[n, i, :])
            temp_idxs.append(i)
        if (np.exp(log_likelihood[n][i] - ent_lnlikelihood[n][i]) < 1-1e-6 ):#and np.exp(log_likelihood[n][i] - ent_lnlikelihood[n][i]) > 1-1e-11):
            temp_array.append((log_likelihood[n][i] - ent_lnlikelihood[n][i]))
            temp_samp_array.append(samples[n, i, :])
            temp_idxs.append(i)
    idxs.append(temp_idxs)
    bad_samples.append(temp_samp_array)
    print(len(temp_array))
    print(temp_idxs)

In [ ]:
for i in n_chain:
    print(i)
    for ii in idxs[i]:
        print(ii)
        print(log_likelihood[i][ii])
        print('\n')

### Recreate likelihood object for every bad step, likelihood should match enterprise exactly since we recreate QuickBurst likelihood object from scratch.
- Print out various likelihood parts for each bad sample
- Perform new run and save relevant likelihood parts for comparison to notebook likelihoods (which match enterprise)
- Compare likelihood attributes at relevant stages.

# OLD CODE IN THIS CELL
### Load in likelihood attributes from QuickBurst MCMC run

### Load in likelihood_attributes and step_array from chain

In [ ]:
pkl_filepath = filepath + '.pkl'
with open(pkl_filepath, 'rb') as f:
    likelihood_attributes = pickle.load(f)

In [ ]:
step_array_filepath = filepath + '_step_array.txt'
print(step_array_filepath)
step_array = np.loadtxt(step_array_filepath, dtype = str)
# step_array = []
# #Getting rid of delimiters that make two or more columns
# with open(step_array_filepath) as fin:
#     full_array = fin.read()
#     print(np.shape(full_array))
#     step_array_temp = full_array.split('\n')
#     for i in range(len(step_array_temp)):
#         step_array.append(step_array_temp[i].split(', '))
# print(step_array[0])
    
# # step_array = np.loadtxt(step_array_filepath, dtype = 'str')
# print(np.shape(step_array[2]))

In [ ]:
np.shape(log_likelihood)

In [ ]:
np.shape(step_array)

## Get attributes for notebook object at each bad step
### 7/27/23: Switching to using dictionaries for purposes of appending to when resuming runs, and accessing later on via key value pairs.  ~ Jacob

In [ ]:
import QuickBurst_lnlike as QB
sanity_check = False
#Generate dictionary with all empty lists for every parameter we care to look at
notebook_attr_dict = {}

attributes = []

##Generate new likehoods for notebook starting here.
bad_QB_likelihoods = []
bad_ent_lnlikelihoods = []

for n in n_chain:
    print('Chain {}'.format(n))
    temp_likelihoods = []
    temp_ent_like = []
    
    #Make list of attributes to look at based on json file
    attributes.append([key for key in likelihood_attributes['chain_{}'.format(n)].keys()])
    
    #Make empty dictionary for all chains
    notebook_attr_dict['chain_{}'.format(n)] = {}
    
    #Make temp dict that has keys for individual chain
    temp_chain_dict = {key: [] for key in likelihood_attributes['chain_{}'.format(n)].keys()}
    
    #Skip chain if no bad samples
    if (len(bad_samples[n]) == 0):
        bad_QB_likelihoods.append(temp_likelihoods)
        bad_ent_lnlikelihoods.append(temp_ent_like)
        continue
        
    #Remake object during each sample, run bad sample through script and check against enterprise
    for i in range(len(bad_samples[n])):
        #Setup object with previous sample
        QB_logl = QB.QuickBurst(pta = pta, psrs = psrs_sim, params = dict(zip(pta.param_names, samples[n, idxs[n][i], 2:])), 
                                Npsr = len(psrs_sim), tref=tref, Nglitch = int(samples[n, idxs[n][i], 1]), Nwavelet = int(samples[n, idxs[n][i], 0]), 
                                Nglitch_max = max_n_glitch ,Nwavelet_max = max_n_wavelet, rn_vary = vary_rn, wn_vary = vary_white_noise, prior_recovery = prior_recovery)
        #Setup next step
        n_wavelet_new = int(samples[n, idxs[n][i], 0])# get_n_wavelet(samples, j, 0)
        n_glitch_new = int(samples[n, idxs[n][i], 1])# get_n_glitch(samples, j, 0)
        bad_step = samples[n, idxs[n][i], 2:]


        temp_likelihoods.append(QB_logl.get_lnlikelihood(bad_step, vary_white_noise = True, vary_red_noise = True))
        #print('Enterprise PTA TNr: ', ent_ptas[n_wavelet_new][n_glitch_new].get_TNr(bad_step))
        temp_ent_like.append(ent_ptas[n_wavelet_new][n_glitch_new].get_lnlikelihood(QB_MCMC.remove_params(bad_step, 0, 0, wavelet_indx, glitch_indx, n_glitch = n_glitch_new, n_wavelet = n_wavelet_new, max_n_glitch = max_n_glitch, max_n_wavelet = max_n_wavelet, params_slice = True)))

        #above should be fine, but occasionaly gets NAN error
        #print('Ent pta params for above glitch/wavelet:', ent_ptas[n_wavelet_new][n_glitch_new].params)

        #Compare attributes for various steps
        for jj in temp_chain_dict.keys():
            #Print statement to check we are comparing apples to apples

            #Save current object attr to dictionary
            #print('Saving {0} to notebook dict for chain {1}'.format(attributes_to_load[jj], n), '\n')
            temp_chain_dict[jj].append(np.copy(getattr(QB_logl, jj)))

    #Save temporary chain dict to first relevant empty dictionary
    notebook_attr_dict['chain_{}'.format(n)] = temp_chain_dict
    print(notebook_attr_dict['chain_{}'.format(n)].keys())
    
    bad_QB_likelihoods.append(temp_likelihoods)
    bad_ent_lnlikelihoods.append(temp_ent_like)
    #first_sample = strip_samples(samples, j, 0, n_wavelet, max_n_wavelet, n_glitch, max_n_glitch)

In [ ]:
pta.get_TNT(dict(zip(pta.param_names, samples[0, idxs[0][0], 2:])))

### Turn ndarray elements into lists to match type of object that is likelihood_attributes

#### For Notebook QB

In [ ]:
for n in range(len(n_chain)):
    print(n)
    #Skip chain if no bad samples
    if (len(bad_samples[n]) == 0):
        bad_QB_likelihoods.append(temp_likelihoods)
        bad_ent_lnlikelihoods.append(temp_ent_like)
        continue
    for jj in attributes[n]:
        #Check if attribute is ndarray
        if str(type(notebook_attr_dict['chain_{}'.format(n)][jj][0])) == "<class 'numpy.ndarray'>":
            #If ndarray, go through all samples and change each element to list
            for kk in range(len(notebook_attr_dict['chain_{}'.format(n)][jj])):
                #Save over ndarray with list type object
                temp_list = notebook_attr_dict['chain_{}'.format(n)][jj][kk].tolist()
                notebook_attr_dict['chain_{}'.format(n)][jj][kk] = temp_list

#### For MCMC QB

In [ ]:
for n in range(len(n_chain)):
    print(n)
    #Skip chain if no bad samples
    if (len(bad_samples[n]) == 0):
        bad_QB_likelihoods.append(temp_likelihoods)
        bad_ent_lnlikelihoods.append(temp_ent_like)
        continue
    for jj in attributes[n]:
        print(jj)
        #Check if attribute is ndarray
        for kk in range(len(notebook_attr_dict['chain_{}'.format(n)][jj])):
            #print(str(type(likelihood_attributes['chain_{}'.format(n)][jj][kk])))
            if str(type(likelihood_attributes['chain_{}'.format(n)][jj][idxs[n][kk]])) == "<class 'numpy.ndarray'>":
                #If ndarray, go through all samples and change each element to list
                #Save over ndarray with list type object
                #temp_list = likelihood_attributes['chain_{}'.format(n)][jj][idxs[n][kk]].tolist()
                likelihood_attributes['chain_{}'.format(n)][jj][idxs[n][kk]] = likelihood_attributes['chain_{}'.format(n)][jj][idxs[n][kk]].tolist()

In [ ]:
type(likelihood_attributes['chain_0']['params'][idxs[0][0]])

# Compare notebook object properties at each step vs MCMC object properties 

In [ ]:
ent_lnlikelihood

### Calculate likelihood difference between resampled bad likelihood points to enterprise likelihoods. These should be 0 (or close to 0).

In [ ]:
ent_like_resample_diff = []
notebook_enterprise_diff = []
QB_like_diff = []
enterprise_like_diff = []
for n in range(len(n_chain)):
    print(n)
    if (len(bad_samples[n]) == 0):
        continue
        
    temp_like_diff = []
    temp_note_diff = []
    temp_QB_diff = []
    temp_ent_diff = []
    
    for i in range(len(bad_samples[n])):
        temp_like_diff.append(bad_QB_likelihoods[n][i] - ent_lnlikelihood[n][idxs[n][i]])
        temp_note_diff.append(bad_QB_likelihoods[n][i] - bad_ent_lnlikelihoods[n][i])
        temp_QB_diff.append(bad_QB_likelihoods[n][i] - log_likelihood[n][idxs[n][i]])
        temp_ent_diff.append(bad_ent_lnlikelihoods[n][i] - ent_lnlikelihood[n][idxs[n][i]])
        
    ent_like_resample_diff.append(temp_like_diff)
    notebook_enterprise_diff.append(temp_note_diff)
    QB_like_diff.append(temp_QB_diff)
    enterprise_like_diff.append(temp_ent_diff)

In [ ]:
#Notebook QB - MCMC ent
ent_like_resample_diff

In [ ]:
#Notebook QB - MCMC QB
QB_like_diff

In [ ]:
#Notebook ent - MCMC ent (should always be 0)
enterprise_like_diff

In [ ]:
for ii in range(len(enterprise_like_diff[0])):
    print('Step taken: ', step_array[idxs[0][ii]], '\n')
    print('Notebook ent, MCMC ent: ', bad_ent_lnlikelihoods[0][ii], ent_lnlikelihood[0][ii], '\n')

In [ ]:
attributes.sort()


### Get difference of all properties of the QB class (notebook - MCMC)

In [ ]:
notebook_attr_dict['chain_{}'.format(0)]['Nvecs'][0]

### Parameter subtraction (Notebook - MCMC)

In [ ]:
# like_property_diff = []
# for n in n_chain:
#     temp_like_diff = []
#     like_property_diff.append(temp_like_diff)

like_property_diff = {}

#Loop through each chain
for n in n_chain:
    temp_diff_dict = {}
    print(n)
    
    temp_diff_dict = {key: [] for key in likelihood_attributes['chain_{}'.format(n)].keys()}
    #print(temp_diff_dict.keys())
    #Check if there are no bad samples for each chain
    if (len(bad_samples[n]) == 0):
        continue
        
    #Loop through all bad samples in chains
    for i in range(len(bad_samples[n])):
        for jj in attributes[n]:   
            
            #If dimensions are greater than 1, take difference for every element for each attribute for each chain sample
            #Also check if object type is list. If either is true, do element wise subtraction.
            if len(np.shape(notebook_attr_dict['chain_{}'.format(n)][jj][i])) > 1 or str(type(notebook_attr_dict['chain_{}'.format(n)][jj][i])) == "<class 'list'>" or str(type(notebook_attr_dict['chain_{}'.format(n)][jj][i])) == "<class 'dict'>":
                #print('Loop! Ms and Ns: ', jj)
                
#                 print(type(notebook_attr_dict['chain_{}'.format(n)][jj][i]))
#                 print(type(likelihood_attributes['chain_{}'.format(n)][jj][idxs[n][i]].tolist()))

                
                #Store each attribute to temp variable
                a = notebook_attr_dict['chain_{}'.format(n)][jj][i]
                b = likelihood_attributes['chain_{}'.format(n)][jj][idxs[n][i]]
            
                #Hardcoded, but if property is a dictionary
                if jj == 'params':
                    #print('yeah:', a.keys())
                    #print('YEAH: ', b.keys())
                    #Create difference in param key value pairs into new temp dict
                    temp_var_dict = {key: a[key] - b[key] for key in a.keys()}
                    #Append temp dict to temp dictionary for set of params at sample i
                    temp_diff_dict[jj].append(temp_var_dict)
                    
                else:
                    #Otherwise, do element wise subtraction for elements in 2D arrays
#                     print(notebook_attr_dict['chain_{}'.format(n)][jj][i])
#                     print(likelihood_attributes['chain_{}'.format(n)][jj][idxs[n][i]])
                    temp_diff_dict[jj].append(np.subtract(list(notebook_attr_dict['chain_{}'.format(n)][jj][i]), list(likelihood_attributes['chain_{}'.format(n)][jj][idxs[n][i]])))
    
            #handles cases of 0d objects     
            else:
                #Otherwise, do element wise subtraction for elements in 2D arrays
                temp_diff_dict[jj].append(np.subtract(notebook_attr_dict['chain_{}'.format(n)][jj][i], likelihood_attributes['chain_{}'.format(n)][jj][idxs[n][i]]))

    like_property_diff['chain_{}'.format(n)] = temp_diff_dict

In [ ]:
np.shape(like_property_diff['chain_0']['wavelet_prm'][0])

### Printing out ratio of elements for each param between QB notebook object and MCMC object

In [ ]:
attributes[0]

## Calculate all likelihood properties ratios

In [ ]:
type(likelihood_attributes['chain_0'][attributes[0][2]][idxs[1][0]])

In [ ]:
notebook_attr_dict['chain_{}'.format(1)][attributes[1][1]][0]

In [ ]:
list(likelihood_attributes['chain_{}'.format(chain_n)][attributes[chain_n][param_check]][idxs[chain_n][check_i]])

In [ ]:
like_property_diff['chain_{}'.format(chain_n)][attributes[chain_n][param_check]][check_i]

In [ ]:
chain_n = 0
check_i = 0
print(samples[chain_n, idxs[chain_n][check_i], 2:])
print(samples[chain_n, idxs[chain_n][check_i]-1, 2:])

In [ ]:
dictionary_ratios = {}

#Loop through each chain
for n in n_chain:
    temp_ratio_dict = {}
    print(n)
    
    #Create temp ratio dictionary for each chain
    temp_ratio_dict = {key: [] for key in likelihood_attributes['chain_{}'.format(n)].keys()}
    
    #Check if there are no bad samples for each chain
    if (len(bad_samples[n]) == 0):
        continue
        
    print(attributes[n])
        
    #Loop through all bad samples in chains
    for i in range(len(bad_samples[n])):
        for jj in attributes[n]:    
            #If dimensions are greater than 1, take difference for every element for each attribute for each chain sample
            #Also check if object type is list. If either is true, do element wise subtraction.
            if len(np.shape(notebook_attr_dict['chain_{}'.format(n)][jj][i])) > 1 or str(type(notebook_attr_dict['chain_{}'.format(n)][jj][i])) == "<class 'list'>" or str(type(notebook_attr_dict['chain_{}'.format(n)][jj][i])) == "<class 'dict'>":
                #Store each attribute to temp variable
                a = like_property_diff['chain_{}'.format(n)][jj][i]
                b = notebook_attr_dict['chain_{}'.format(n)][jj][i]

                #Hardcoded, but if property is a dictionary
                if jj == 'params':
                    #print('params print: ', jj)
                    #Create difference in param key value pairs into new temp dict
                    temp_var_dict = {key: a[key]/b[key] for key in a.keys()}

                    #Append temp dict to temp dictionary for set of params at sample i
                    temp_ratio_dict[jj].append(temp_var_dict)

                elif jj == 'wavelet_prm':
                    #print('wavelet_prm print: ', jj)
                    #do stuff
                    #print('types: ', type(like_property_diff['chain_{}'.format(n)][jj][i]), type(notebook_attr_dict['chain_{}'.format(n)][jj][i]))
                    temp_wave_prm = np.array(notebook_attr_dict['chain_{}'.format(n)][jj][i])
                    temp_ratio_dict[jj].append(np.divide(like_property_diff['chain_{}'.format(n)][jj][i], temp_wave_prm))
                    
                else:
                    #print('else print: ', jj)
                    #Otherwise, do element wise subtraction for elements in 2D arrays
                    temp_ratio_dict[jj].append(np.divide(like_property_diff['chain_{}'.format(n)][jj][i], notebook_attr_dict['chain_{}'.format(n)][jj][i]))
            else:
                #Otherwise, do element wise subtraction for elements in 0D arrays
                temp_ratio_dict[jj].append(np.divide(like_property_diff['chain_{}'.format(n)][jj][i], notebook_attr_dict['chain_{}'.format(n)][jj][i]))
                    
    dictionary_ratios['chain_{}'.format(n)] = temp_ratio_dict

In [ ]:
attributes[0]

In [ ]:
chain_n = 0
param_check = 4
check_index = 0

In [ ]:
idxs[chain_n][check_index]

In [ ]:
step_array[idxs[chain_n][check_index]]

In [ ]:
likelihood_attributes['chain_{}'.format(chain_n)][attributes[chain_n][param_check]][idxs[chain_n][check_index]]

In [ ]:
likelihood_attributes['chain_{}'.format(chain_n)][attributes[chain_n][param_check]][idxs[chain_n][check_index]-1]

In [ ]:
dictionary_ratios['chain_{}'.format(chain_n)][attributes[chain_n][param_check]][check_index])

In [ ]:
MMrow = 7
like_property_diff[2][0][0][MMrow]/notebook_attr_dict[chain_var_names[2][0]][0][MMrow]

#############################################################

In [ ]:
NN = 500

rn = True
wn = False

log_L_Ent = []
log_L_Fast = []
accept = []
#amplitudes = []
d0 = parameter.sample(pta_sim.params)
d0_previous = d0.copy()
x0 = np.array([d0[par.name] for par in pta_sim.params])
FB1.get_lnlikelihood(x0, vary_white_noise = wn, vary_red_noise = rn)
FB1.save_values(accept_new_step=True)
for n in range(NN):
    #amplitudes.append(i*0.5e-6)
    if 0.5 < np.random.uniform():
        #print('normal step')
        d0 = parameter.sample(pta_sim.params)
        x0 = np.array([d0[par.name] for par in pta_sim.params])
        log_L_Ent.append(pta_sim.get_lnlikelihood(x0))
        log_L_Fast.append(FB1.get_lnlikelihood(x0, vary_white_noise = wn, vary_red_noise = rn))#, no_step = True))
    else:
        #print('fast step')
        d0_2 = parameter.sample(pta_sim.params)
        for i in range(len(psrs_sim)):
            for k,par in enumerate(pta_sim.param_names):
                if psrs_sim[i].name in par: #set perpulsar terms to match
                    if 'ecorr' in par or 'efac' in par or 'equad' in par or 'log10_A' in par or 'gamma' in par:
                        d0_2[pta_sim.param_names[k]] = d0_previous[pta_sim.param_names[k]]
                elif 'gw_gamma' in par or 'gw_log10_A' in par:#match common process
                    d0_2[pta_sim.param_names[k]] = d0_previous[pta_sim.param_names[k]]
        for nn in range(N_glitches):
            d0_2['Glitch_'+str(nn)+'_log10_f0'] = d0_previous['Glitch_'+str(nn)+'_log10_f0']
            d0_2['Glitch_'+str(nn)+'_psr_idx'] = d0_previous['Glitch_'+str(nn)+'_psr_idx']
            d0_2['Glitch_'+str(nn)+'_t0'] = d0_previous['Glitch_'+str(nn)+'_t0']
            d0_2['Glitch_'+str(nn)+'_tau'] = d0_previous['Glitch_'+str(nn)+'_tau']
        for ww in range(N_wavelets):
            d0_2['wavelet_'+str(ww)+'_log10_f0'] = d0_previous['wavelet_'+str(ww)+'_log10_f0']
            d0_2['wavelet_'+str(ww)+'_t0'] = d0_previous['wavelet_'+str(ww)+'_t0']
            d0_2['wavelet_'+str(ww)+'_tau'] = d0_previous['wavelet_'+str(ww)+'_tau']
        #print('d0: ',d0)
        #print('d0_2 modified: ',d0_2)
        #d0_2 = d0
        x0_2 = np.array([d0_2[par.name] for par in pta_sim.params])
        log_L_Ent.append(pta_sim.get_lnlikelihood(x0_2))
        log_L_Fast.append(FB1.get_lnlikelihood(x0_2, vary_white_noise = False, vary_red_noise = False))
    if 0.5 < np.random.uniform():
        FB1.save_values(accept_new_step=False)
        accept.append('No')
        d0 = d0_previous.copy()
    else:
        FB1.save_values(accept_new_step=True)
        accept.append('Yes')
        d0_previous = d0.copy()
    #print('run ',n)
log_L_Ent = np.array(log_L_Ent)
log_L_Fast = np.array(log_L_Fast)
accept = np.array(accept)

In [ ]:
pta_sim.param_names

In [ ]:
'Glitch_4_log10_f0': -7.227727184731241, 'Glitch_4_log10_h': -9.184254001093874, 'Glitch_4_phase0': 5.126291443469657, 'Glitch_4_psr_idx': 11.06653403004687, 'Glitch_4_t0': 6.781043649900989, 'Glitch_4_tau': 1.012983761027325, 'JPSR00_efac': 5.7865257197768125, 'JPSR00_log10_t2equad': -5.458430039323788, 'JPSR01_efac': 7.408289003479412, 'JPSR01_log10_t2equad': -7.386667862884638, 'JPSR02_efac': 5.694954690794321, 'JPSR02_log10_t2equad': -7.1875982388319795, 'JPSR03_efac': 4.356071771472992, 'JPSR03_log10_t2equad': -5.212509749153512, 'JPSR04_efac': 8.003935924228779, 'JPSR04_log10_t2equad': -8.043245424082997, 'JPSR05_efac': 3.4396696312250334, 'JPSR05_log10_t2equad': -7.09399898317015
'Glitch_4_log10_f0': -7.227727184731241, 'Glitch_4_log10_h': -9.120508561286098, 'Glitch_4_phase0': 0.6447336458972177, 'Glitch_4_psr_idx': 11.06653403004687, 'Glitch_4_t0': 6.781043649900989, 'Glitch_4_tau': 1.012983761027325, 'JPSR00_efac': 5.7865257197768125, 'JPSR00_log10_t2equad': -5.458430039323788, 'JPSR01_efac': 7.408289003479412, 'JPSR01_log10_t2equad': -7.386667862884638, 'JPSR02_efac': 5.694954690794321, 'JPSR02_log10_t2equad': -7.1875982388319795, 'JPSR03_efac': 4.356071771472992, 'JPSR03_log10_t2equad': -5.212509749153512, 'JPSR04_efac': 8.003935924228779, 'JPSR04_log10_t2equad': -8.043245424082997, 'JPSR05_efac': 3.4396696312250334, 'JPSR05_log10_t2equad': -7.09399898317015

In [ ]:
print(log_L_Ent[10:30])
print(log_L_Fast[10:30])
print(accept[10:30])

In [ ]:
plt.plot(log_L_Ent, log_L_Ent, ls='--', marker='.', color='xkcd:blue', label = 'Ent')
plt.plot(log_L_Ent, log_L_Fast, ls='', marker='.', color='xkcd:green', label = 'FastB')
plt.legend(loc='upper left')
plt.xlabel("ent log(like)")
plt.ylabel("fast log(likelihood)")
plt.title('lnlikelihood comparisons multi glitch on sim')
#plt.savefig('/home/reyna/BayesHopperBurst/testing plots/simulated_multi_transient_like_comp.png')

In [ ]:
np.max(np.exp(log_L_Fast - log_L_Ent))
temp_array = []
idxs = []
for i in range(len(log_L_Fast)):
    if np.exp(log_L_Fast[i] - log_L_Ent[i]) > 1e2:
        temp_array.append(np.exp(log_L_Fast[i] - log_L_Ent[i]))
        idxs.append(i)
    

In [ ]:
plt.plot(np.exp(log_L_Fast - log_L_Ent), ls='', marker='.', color='xkcd:blue')
#plt.gca().axhline(0.1, ls='--', color='xkcd:green')
plt.xlabel("index")
plt.ylabel("delta log(likelihood)")
#plt.ylim(bottom = 1-1e-9, top = 1+1e-9)
plt.title('dif for sim random samples')
#plt.savefig('/home/reyna/BayesHopperBurst/testing plots/simulated_Multi_transient_precentDif.png')

In [ ]:
np.exp(log_L_Fast - log_L_Ent)

In [ ]:
plt.plot(log_L_Fast/log_L_Ent - 1, ls='', marker='.', color='xkcd:blue')
#plt.gca().axhline(0.1, ls='--', color='xkcd:green')
plt.xlabel("index")
plt.ylabel("delta log(likelihood)")
plt.title('%dif for randome samples')
#plt.savefig('/home/reyna/BayesHopperBurst/testing plots/simulated_Multi_transient_precentDif.png')

In [ ]:
plt.plot(amplitudes, log_L_Ent, ls='--', marker='.', color='xkcd:green', label = 'Ent')
plt.plot(amplitudes, log_L_Fast, ls='', marker='.', color='xkcd:blue', label = 'FastB')
plt.legend(loc='upper right')
plt.xlabel("amp")
plt.ylabel("log(likelihood)")
plt.title('lnlikelihood vs amplitude simulated single transient')
#plt.savefig('/home/reyna/BayesHopperBurst/testing plots/simulated_transient_like_VS_amp.png')

In [ ]:
plt.plot(amplitudes, log_L_Fast-log_L_Ent, ls='', marker='.', color='xkcd:blue')
#plt.gca().axhline(0.1, ls='--', color='xkcd:green')
plt.xlabel("amp")
plt.ylabel("delta log(likelihood)")
#plt.yscale('log')

In [ ]:
plt.plot(amplitudes, log_L_Fast/log_L_Ent - 1., ls='', marker='.', color='xkcd:blue')
#plt.gca().axhline(0.1, ls='--', color='xkcd:green')
plt.xlabel("amp")
plt.ylabel("precent diff log(likelihood)")
plt.title('%dif vs amplitude simulated single transient')
plt.savefig('/home/reyna/BayesHopperBurst/testing plots/simulated_transient_precentDif_VS_amp.png')
#plt.yscale('log')

In [ ]:
for i, psr in enumerate(psrs_sim):
    plt.figure(i)
    plt.plot(psr.toas/86400, psr.residuals, ls='', marker='.')
    plt.plot(psr.toas/86400, pta_sim.get_delay(d0)[i],color='xkcd:red')
    
    f0 = 10**(x0[0])
    tau = (365.25*24*3600)*x0[5]
    t0 = (365.25*24*3600)*x0[4]
    
    Cosine = np.exp(-1*((FB1.toas[i] - t0)/tau)**2)*np.cos(2*np.pi*f0*(FB1.toas[i] - t0))
    Sine = np.exp(-1*((FB1.toas[i] - t0)/tau)**2)*np.sin(2*np.pi*f0*(FB1.toas[i] - t0))
    print(FB1.sigma)
    fast_delay = FB1.sigma[0]*Cosine + FB1.sigma[1]*Sine
    plt.plot(psr.toas/86400, fast_delay- pta_sim.get_delay(d0)[i] , ls='--',color='xkcd:green') 

In [ ]:
for i, psr in enumerate(psrs_sim):
    plt.figure(i)
    #plt.plot(psr.toas/86400, psr.residuals, ls='', marker='.')
    #plt.plot(psr.toas/86400, pta_sim.get_delay(d0)[i],color='xkcd:red')
    
    f0 = 10**(x0[0])
    tau = (365.25*24*3600)*x0[5]
    t0 = (365.25*24*3600)*x0[4]
    
    Sigma0 = 10**(x0[1])*np.cos(x0[2]) #10**(x0[1]), x0[2]
    Sigma1 = -10**(x0[1])*np.sin(x0[2]) #10**(x0[1]), x0[2]
    
    Cosine = np.exp(-1*((FB1.toas[i] - t0)/tau)**2)*np.cos(2*np.pi*f0*(FB1.toas[i] - t0))
    Sine = np.exp(-1*((FB1.toas[i] - t0)/tau)**2)*np.sin(2*np.pi*f0*(FB1.toas[i] - t0))
    print(FB1.sigma)
    fast_delay = Sigma0*Cosine + Sigma1*Sine #FB1.sigma[0]*Cosine + FB1.sigma[1]*Sine
    plt.plot(psr.toas/86400, fast_delay - pta_sim.get_delay(d0)[i], ls='-',color='xkcd:green') 
    #plt.plot(psr.toas/86400, FB1.sigma[0]*Cosine , ls='--',color='xkcd:blue') 
    #plt.plot(psr.toas/86400, FB1.sigma[1]*Sine , ls='--',color='xkcd:red') 